# Rasterize Natural Gas Pipeline Network
Creates a raster version at the same extent and cell size of the MIT cost surface.

### Workflow
* Convert feature class to geopandas dataframe
* Subset NC records 
* Group values on `STATUS`, `TYPE`, and `DIAMETER`
 * This results in 29 unique combinations. Each combination is given a unique ID (1 thru 29)
* Add a new field to the list of unique combinations: 2 ^^ index

In [ ]:
import rasterio
from rasterio import features
import geopandas as gpd
import pandas as pd
import numpy as np

#### Import the REXTAG natural gas feature class
*NOTE: This is proprietary dataset stored on the secure server*

In [ ]:
#Read in features
pipelines_gdb = 'C:\\Workspace\\Rextag_2July2020\\Rextag_Zone2_Natural_Gas.gdb'
pipelines_lyr = 'NaturalGasPipelines'

In [ ]:
#Read in pipeline dataset as a geodataframe
gdf = gpd.read_file(pipelines_gdb,driver='FileGDB',layer=pipelines_lyr)

In [ ]:
#Print number of unique values in each - ENTIRE DATASET
print(F"STATE:\t\t{gdf.STATE_NAME.nunique()}")
print(F"STATUS:\t\t{gdf.STATUS.nunique()}")
print(F"TYPE:\t\t{gdf.TYPE.nunique()}")
print(F"DIAMETER:\t{gdf.DIAMETER.nunique()}")

In [ ]:
#Group the data
grp = gdf.groupby(['STATE_NAME','TYPE','STATUS','DIAMETER'])

In [ ]:
#Output to CSV file
grp.count().to_csv("ALL_stats.csv")

## Isolate and map NC data

In [ ]:
#Isolate NC records
gdf_nc = gdf.loc[gdf['STATE_NAME']=='North Carolina'].reset_index()

In [ ]:
#Group on attribs
grp_nc = gdf_nc.groupby(['STATUS','TYPE','DIAMETER'])

#Output grouped values to a dataframe
df_unique = grp_nc.index.count().reset_index().drop('index',axis=1)
#Save the index value to a field
df_unique['UNIQIUE_ID'] = df_unique.index

#Compute 2^^ index value
df_unique['OUT_CODE'] = df_unique['UNIQIUE_ID'].apply(lambda x: 2**x)

#Save to file
df_unique.to_csv('Data_Lookup.csv',index=False)

In [ ]:
#Join the OUTCODE to each row in the gdf_dataset
gdf_nc2 = pd.merge(left = gdf_nc, left_on = ['STATUS','TYPE','DIAMETER'],
                   right = df_unique, right_on = ['STATUS','TYPE','DIAMETER'],
                   how = 'left'
                  )

In [ ]:
gdf_nc2.columns

In [ ]:
gdf_nc2.iloc[:5,[7,8,9,-1]]

### Now rasterize each unique value into an image layer, then add all layers

#### Import a 100 m template raster
This is used to extract the extent and cell size of the created pipeline raster

In [ ]:
#Load the cost surface raster
template_raster_dataset = rasterio.open('../data/processed/PipeDiameter_100m.tif')
#Get the surface 
template_raster_ = template_raster_dataset.read(1)
#Extract the coordinate reference system
raster_crs = template_raster_dataset.crs
#Get the metadata
raster_meta = template_raster_dataset.meta

In [ ]:
#Transform to same crs as cost surface
gdf_nc2 = gdf_nc2.to_crs(raster_crs)

#### Rastersize each unique OUTPUT_CODE as a unique layer

In [ ]:
#Create a list to store each layer
image_list = []
#Set the name of the field containing the unique values
fldName ='OUT_CODE'
#Iterate through each unique value
for val in df_unique[fldName].unique():
    print(val)
    #Subset the data
    gdf_subset = gdf_nc2.loc[gdf_nc2[fldName] == val]
    #Get the geometries from the subset
    out_shapes = zip(gdf_subset.geometry,gdf_subset[fldName])
    #Rasterize the geometries
    image = features.rasterize(shapes=out_shapes,
                               out_shape=template_raster_dataset.shape,
                               transform=template_raster_dataset.transform,
                               all_touched=False)
    image_list.append(image)

In [ ]:
#Stack images and add together
img_stack = np.stack(image_list)

In [ ]:
img_stack.shape

In [ ]:
img_sum = img_stack.sum(axis=0)
img_sum.shape

In [ ]:
#Get the bit depth
min_dtype = rasterio.dtypes.get_minimum_dtype(gdf_nc2[fldName])
#Update the metadata
cost_meta.update(
{'driver':'GTiff',
 'nodata':0,
 'dtype':min_dtype
})
#Export as tiff
with rasterio.open('../data/processed/image_stack.tif','w',**cost_meta) as dst:
    dst.write(img_sum, indexes=1)


#### Explort the result to a GeoTIFF

In [ ]:
min_dtype

In [ ]:
foo = image_list[0] | image_list[1]|image_list[1]

In [ ]:
foo.max()

In [ ]:
#Update metadata for output
cost_meta = cost_surface_dataset.meta

In [ ]:
cost_meta.update(
    {'driver':'GTiff',
     'nodata':255,
     'dtype':min_dtype
    })

In [ ]:
#Export as tiff
with rasterio.open('..\\data\\processed\\USA_pipes_operational.tif','w',**cost_meta) as dst:
    dst.write(image, indexes=1)